In [7]:
import pandas as pd
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm
from typing import List, Union, Optional, Callable
import pickle
from Bio import AlignIO, SeqIO
from ete3 import Tree, TreeNode
from gctree import CollapsedTree
import nltk
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import umap
from ete3 import Tree, faces, TreeStyle, NodeStyle, TextFace, SequenceFace, COLOR_SCHEMES, CircleFace
from GCTree_preparation import *
import warnings
import math
warnings.filterwarnings("ignore")

path_to_storage = "/media/hieunguyen/HNHD01/storage/all_BSimons_datasets"
outdir = "/media/hieunguyen/GSHD_HN01/outdir/sc_bulk_BCR_data_analysis_v0.1"

PROJECT = "240826_BSimons"
sc_PROJECT = "240805_BSimons"

path_to_main_output = f"{outdir}/tree_analysis/{PROJECT}"
path_to_01_output = os.path.join(path_to_main_output, "01_output")
os.system(f"mkdir -p {path_to_01_output}")

output_type = "all"

path_to_trees = os.path.join(path_to_storage, PROJECT, "GCtrees/v0.2", output_type)

all_tree_folder = [item for item in pathlib.Path(path_to_trees).glob("*/*") if 
                   os.path.isfile(f"{str(item)}/02_dnapars/gctree.out.inference.1.nk") == True]

all_nk_files = [item for item in pathlib.Path(path_to_trees).glob("*/*/*gctree.out.inference.1.nk")]  
print(f"Number of trees: {len(all_tree_folder)}")   

path_to_04_output = os.path.join(outdir, "VDJ_output", "04_output")
thres = 0.85

mid_metadata = pd.read_excel("/media/hieunguyen/HNSD01/src/sc_bulk_BCR_data_analysis/preprocessing/240826_BSimons/240829 sample sheet.xlsx")
mid_metadata.columns = ["MID", "mouse", "organ", "population"]

##### load tree object
maindf = pd.read_csv(f"{path_to_01_output}/tree_summarydf.csv")
    # Reload the dictionary from the pickle file
with open(f"{path_to_01_output}/saveTreeobj.pkl", "rb") as f:
    saveTreeobj = pickle.load(f)

##### load tree summary df
maindf = pd.read_csv(f"{path_to_01_output}/tree_summarydf.addedAbundance.csv")

##### load distance df
distdf = pd.read_csv(os.path.join(path_to_01_output, f"{PROJECT}_distdf.csv"))
distdf["V_gene"] = distdf["cloneID"].apply(lambda x: "-".join(x.split("_")[1].split("-")[0:2]))
distdf["J_gene"] = distdf["cloneID"].apply(lambda x: "-".join(x.split("_")[2].split("-")[0:1]))
distdf["CDR3_len"] = distdf["CDR3nt"].apply(lambda x: len(x))
##### load single cell clone df
scdf = pd.read_csv(os.path.join(f"{outdir}/GEX_output/11_output", f"{sc_PROJECT}.csv"), index_col = [0])
scdf["CDR3_len"] = scdf["nSeqCDR3"].apply(lambda x: len(x))

scdf.head()

Number of trees: 1178


,barcode,V.gene,J.gene,aaSeqCDR3,nSeqCDR3,VJseq.combi,VJ.combi,VJ.len.combi,VJcombi_CDR3_0.85,CDR3_len
1,M1_AAACCTGCACGTGAGA-1,IGHV1-66,IGHJ4,CARGAMDYW,TGTGCAAGAGGGGCTATGGACTACTGG,IGHV1-66_IGHJ4_CARGAMDYW_TGTGCAAGAGGGGCTATGGAC...,IGHV1-66_IGHJ4,IGHV1-66_IGHJ4_27,IGHV1-66_IGHJ4_27_1,27
2,M1_AAACGGGAGTTCGCAT-1,IGHV1-66,IGHJ2,CAINSYRKGELGYW,TGTGCAATAAACAGCTATCGTAAAGGGGAACTGGGCTACTGG,IGHV1-66_IGHJ2_CAINSYRKGELGYW_TGTGCAATAAACAGCT...,IGHV1-66_IGHJ2,IGHV1-66_IGHJ2_42,IGHV1-66_IGHJ2_42_1,42
3,M1_AAAGATGGTTAAAGTG-1,IGHV1-75,IGHJ4,CARSPLLTMAYW,TGTGCAAGATCCCCTTTACTCACTATGGCCTACTGG,IGHV1-75_IGHJ4_CARSPLLTMAYW_TGTGCAAGATCCCCTTTA...,IGHV1-75_IGHJ4,IGHV1-75_IGHJ4_36,IGHV1-75_IGHJ4_36_1,36
4,M1_AAAGATGTCGATCCCT-1,IGHV8-8,IGHJ2,CARIEGLGRRYFDYW,TGTGCTCGAATAGAGGGACTGGGACGAAGGTACTTTGACTACTGG,IGHV8-8_IGHJ2_CARIEGLGRRYFDYW_TGTGCTCGAATAGAGG...,IGHV8-8_IGHJ2,IGHV8-8_IGHJ2_45,IGHV8-8_IGHJ2_45_1,45
5,M1_AAAGATGTCGCCTGTT-1,IGHV5-4,IGHJ4,CGSYYYGSSRYYYAMDYW,TGTGGGAGTTATTACTACGGTAGTAGCCGATATTACTATGCTATGG...,IGHV5-4_IGHJ4_CGSYYYGSSRYYYAMDYW_TGTGGGAGTTATT...,IGHV5-4_IGHJ4,IGHV5-4_IGHJ4_54,IGHV5-4_IGHJ4_54_1,54


In [2]:
distdf.head()

,seqid,MID,mouseid,cloneID,CDR3nt,CDR3aa,rootness,topo_rootness,dist_to_deepest,topo_dist_to_deepest,...,topo_leafness,furthest_child_node,leafness,num_node,num_leaves,sample_type,mix_node,V_gene,J_gene,CDR3_len
0,seq1,MID2_MID3,m1,m1_IGHV1-64-01_IGHJ2-01_36_3,TGTGCAAGATGGGAACTATGGACTCTTGACTACTGG,CARWELWTLDYW,37.0,1.0,7.0,1.0,...,0.0,seq3,7.0,4,2,m1 SI mid_m1 SI prox,yes,IGHV1-64,IGHJ2,36
1,seq2,MID4,m1,m1_IGHV1-64-01_IGHJ2-01_36_3,TGTGCAAGATGGGAACTATGGGCTCTTGACTACTGG,CARWELWALDYW,38.0,2.0,8.0,1.0,...,0.0,seq2,0.0,4,2,m1 colon,no,IGHV1-64,IGHJ2,36
2,seq3,MID4,m1,m1_IGHV1-64-01_IGHJ2-01_36_3,TGTGTAAGATGGGAACTATGGGCTCTTGACTACTGG,CVRWELWALDYW,44.0,2.0,0.0,0.0,...,0.0,seq3,0.0,4,2,m1 colon,no,IGHV1-64,IGHJ2,36
3,seq1,MID2_MID4,m1,m1_IGHV1-84-01_IGHJ3-01_42_1,TGTGCAAGGGGGCTAACTGGGACGTCCTGGTTTGCTTACTGG,CARGLTGTSWFAYW,30.0,2.0,59.0,1.0,...,0.0,seq2,59.0,9,5,m1 SI mid_m1 colon,yes,IGHV1-84,IGHJ3,42
4,seq2,MID4,m1,m1_IGHV1-84-01_IGHJ3-01_42_1,TGTGCAAGGGGGCTAACTGGGACGTCCTGGTTTGCTTACTGG,CARGLTGTSWFAYW,89.0,3.0,0.0,0.0,...,0.0,seq2,0.0,9,5,m1 colon,no,IGHV1-84,IGHJ3,42


In [ ]:
i = 1
cloneid = distdf.iloc[i]["cloneID"]
v_gene = distdf.iloc[i]["V_gene"]
j_gene = distdf.iloc[i]["J_gene"]
cdr3_length = distdf.iloc[i]["CDR3_len"]
cdr3_seq = distdf.iloc[1]["CDR3nt"]
seqid = distdf.iloc[1]["seqid"]
tmpdf_scdf = scdf[(scdf["V.gene"] == v_gene) & (scdf["J.gene"] == j_gene) & (scdf["CDR3_len"] == cdr3_length)]

if (tmpdf_scdf.shape[0] != 0):
    tmpdf_scdf["bulk_CDR3nt"] = cdr3_seq
    tmpdf_scdf["countDiff_to_bulk_CDR3nt"] = tmpdf_scdf[["nSeqCDR3", "bulk_CDR3nt"]].apply(
        lambda x: nltk.edit_distance(x[0], x[1]), axis = 1
    )
    tmpdf_scdf["dist_to_bulk_CDR3nt"] = tmpdf_scdf[["nSeqCDR3", "bulk_CDR3nt"]].apply(
        lambda x: nltk.edit_distance(x[0], x[1]) / min(len(x[0]), len(x[1])), axis = 1
    )
    tmpdf_scdf["cloneID"] = cloneid
    tmpdf_scdf["seqid"] = 

In [12]:
tmpdf_scdf

,barcode,V.gene,J.gene,aaSeqCDR3,nSeqCDR3,VJseq.combi,VJ.combi,VJ.len.combi,VJcombi_CDR3_0.85,CDR3_len,bulk_CDR3nt,countDiff_to_bulk_CDR3nt,dist_to_bulk_CDR3nt
1279,M2_GGGAATGCATAAGACA-1,IGHV1-64,IGHJ2,CARSGAGTEDYW,TGTGCAAGAAGTGGGGCTGGGACGGAGGACTACTGG,IGHV1-64_IGHJ2_CARSGAGTEDYW_TGTGCAAGAAGTGGGGCT...,IGHV1-64_IGHJ2,IGHV1-64_IGHJ2_36,IGHV1-64_IGHJ2_36_1,36,TGTGCAAGATGGGAACTATGGGCTCTTGACTACTGG,11,0.305556
1746,P1_ACGGAGATCGTGACAT-1,IGHV1-64,IGHJ2,CARWTAQATDYW,TGTGCAAGATGGACAGCTCAGGCTACAGACTACTGG,IGHV1-64_IGHJ2_CARWTAQATDYW_TGTGCAAGATGGACAGCT...,IGHV1-64_IGHJ2,IGHV1-64_IGHJ2_36,IGHV1-64_IGHJ2_36_2,36,TGTGCAAGATGGGAACTATGGGCTCTTGACTACTGG,9,0.250000
1981,P1_CATCGGGAGTGGGCTA-1,IGHV1-64,IGHJ2,CARGGYWDCDYW,TGTGCAAGAGGGGGGTACTGGGACTGTGACTACTGG,IGHV1-64_IGHJ2_CARGGYWDCDYW_TGTGCAAGAGGGGGGTAC...,IGHV1-64_IGHJ2,IGHV1-64_IGHJ2_36,IGHV1-64_IGHJ2_36_3,36,TGTGCAAGATGGGAACTATGGGCTCTTGACTACTGG,8,0.222222
2627,P1_TAGTGGTGTTATCGGT-1,IGHV1-64,IGHJ2,CARYWENYFDYW,TGTGCAAGATACTGGGAAAACTACTTTGACTACTGG,IGHV1-64_IGHJ2_CARYWENYFDYW_TGTGCAAGATACTGGGAA...,IGHV1-64_IGHJ2,IGHV1-64_IGHJ2_36,IGHV1-64_IGHJ2_36_4,36,TGTGCAAGATGGGAACTATGGGCTCTTGACTACTGG,10,0.277778
2929,P2_ACGCCGATCCGTTGCT-1,IGHV1-64,IGHJ2,CARGYDGYLDYW,TGTGCAAGAGGGTATGATGGTTACCTTGACTACTGG,IGHV1-64_IGHJ2_CARGYDGYLDYW_TGTGCAAGAGGGTATGAT...,IGHV1-64_IGHJ2,IGHV1-64_IGHJ2_36,IGHV1-64_IGHJ2_36_5,36,TGTGCAAGATGGGAACTATGGGCTCTTGACTACTGG,8,0.222222
3023,P2_ATGCGATCATCTCCCA-1,IGHV1-64,IGHJ2,CARKGLLRFDYW,TGTGCAAGAAAGGGGTTACTACGCTTTGACTACTGG,IGHV1-64_IGHJ2_CARKGLLRFDYW_TGTGCAAGAAAGGGGTTA...,IGHV1-64_IGHJ2,IGHV1-64_IGHJ2_36,IGHV1-64_IGHJ2_36_6,36,TGTGCAAGATGGGAACTATGGGCTCTTGACTACTGG,9,0.250000
3424,P2_GTAACGTGTTACCAGT-1,IGHV1-64,IGHJ2,CARLLSHYFDYW,TGTGCAAGACTGCTATCCCACTACTTTGACTACTGG,IGHV1-64_IGHJ2_CARLLSHYFDYW_TGTGCAAGACTGCTATCC...,IGHV1-64_IGHJ2,IGHV1-64_IGHJ2_36,IGHV1-64_IGHJ2_36_7,36,TGTGCAAGATGGGAACTATGGGCTCTTGACTACTGG,11,0.305556
